In [1]:
# spark 세션 생성

from pyspark.sql import SparkSession
from pyspark.sql import types as T
from pyspark.sql import functions as F

spark = SparkSession.builder \
    .appName("tutorial") \
    .getOrCreate()

In [3]:
# 기본 파티션 용량 알아보기!

# print(spark.conf.get("spark.sql.files.maxPartitionBytes"))

value = int(spark.conf.get("spark.sql.files.maxPartitionBytes")[:-1])
print(f"{value / (1024 * 1024)} MB")

128.0 MB


In [4]:
# 기본 병렬처리 개수 알아보기!

spark.sparkContext.defaultParallelism

2

In [7]:
# 기본 파티션 용량 변경

spark.conf.set("spark.sql.files.maxPartitionBytes", 1024 * 1024 * 500)

value = int(spark.conf.get("spark.sql.files.maxPartitionBytes")[:-1])
print(f"{value / (1024 * 1024)} MB")

50.0 MB


## 실습

In [8]:
import time

# Case 1: 파티션을 너무 잘게 쪼갰을 때

df = spark.range(0, 10000000)
# 1000만 건을 1000개 파티션으로 쪼갬
small_part_df = df.repartition(1000)

start = time.time()
small_part_df.count() # 단순 세기 작업
print(f"파티션 : {time.time() - start:.2f} 초")

파티션 : 30.13 초


In [9]:
df.rdd.getNumPartitions()

2

In [10]:
# Case 2: 파티션을 너무 크게 잡았을 때 (병렬성 포기)

# 1개 파티션
large_part_df = df.coalesce(1)

start = time.time()
large_part_df.count()
print(f"파티션 1개(과소): {time.time() - start:.2f} 초")

파티션 1개(과소): 1.82 초


In [11]:
# Case 3: 적절한 파티션
best_part_df = df.repartition(6)

start = time.time()
best_part_df.count()
print(f"파티션 6개(적절): {time.time() - start:.2f} 초")

파티션 6개(적절): 5.68 초
